In [ ]:
import os
from collections import Counter
import logging
import sys
from pathlib import Path
import subprocess
import os
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import IPython
import pandas as pd
from tqdm import tqdm

from dotenv import load_dotenv

In [ ]:
def find_comp_gen_dir():
    """Find the computational_genetic_genealogy directory by searching up from current directory."""
    current = Path.cwd()
    
    # Search up through parent directories
    while current != current.parent:
        # Check if target directory exists in current path
        target = current / 'computational_genetic_genealogy'
        if target.is_dir():
            return target
        # Move up one directory
        current = current.parent
    
    raise FileNotFoundError("Could not find computational_genetic_genealogy directory")

def load_env_file():
    """Find and load the .env file from the computational_genetic_genealogy directory."""
    try:
        # Find the computational_genetic_genealogy directory
        comp_gen_dir = find_comp_gen_dir()
        
        # Look for .env file
        env_path = comp_gen_dir / '.env'
        if not env_path.exists():
            print(f"Warning: No .env file found in {comp_gen_dir}")
            return None
        
        # Load the .env file
        load_dotenv(env_path, override=True)
        print(f"Loaded environment variables from: {env_path}")
        return env_path
        
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return None

# Use the function
env_path = load_env_file()

In [ ]:
working_directory = os.getenv('PROJECT_WORKING_DIR', default=None)
data_directory = os.getenv('PROJECT_DATA_DIR', default=None)
references_directory = os.getenv('PROJECT_REFERENCES_DIR', default=None)
results_directory = os.getenv('PROJECT_RESULTS_DIR', default=None)
utils_directory = os.getenv('PROJECT_UTILS_DIR', default=None)

print(f"Working Directory: {working_directory}")
os.environ["WORKING_DIRECTORY"] = working_directory
print(f"Data Directory: {data_directory}")
os.environ["DATA_DIRECTORY"] = data_directory
print(f"References Directory: {references_directory}")
os.environ["REFERENCES_DIRECTORY"] = references_directory
print(f"Results Directory: {results_directory}")
os.environ["RESULTS_DIRECTORY"] = results_directory
print(f"Utils Directory: {utils_directory}")
os.environ["UTILS_DIRECTORY"] = utils_directory

os.chdir(working_directory)
print(f"The current directory is {os.getcwd()}")

In [ ]:
def configure_logging(log_filename, log_file_debug_level="INFO", console_debug_level="INFO"):
    """
    Configure logging for both file and console handlers.

    Args:
        log_filename (str): Path to the log file where logs will be written.
        log_file_debug_level (str): Logging level for the file handler.
        console_debug_level (str): Logging level for the console handler.
    """
    # Create a root logger
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)  # Capture all messages at the root level

    # Convert level names to numeric levels
    file_level = getattr(logging, log_file_debug_level.upper(), logging.INFO)
    console_level = getattr(logging, console_debug_level.upper(), logging.INFO)

    # File handler: Logs messages at file_level and above to the file
    file_handler = logging.FileHandler(log_filename)
    file_handler.setLevel(file_level)
    file_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(file_formatter)

    # Console handler: Logs messages at console_level and above to the console
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setLevel(console_level)
    console_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(console_formatter)

    # Add handlers to the root logger
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
    
def clear_logger():
    """Remove all handlers from the root logger."""
    logger = logging.getLogger()
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)
        
log_filename = os.path.join(results_directory, "labX_log.txt")
print(f"The Lab X log file is located at {log_filename}.")

# Ensure the results_directory exists
if not os.path.exists(results_directory):
    os.makedirs(results_directory)

# Check if the file exists; if not, create it
if not os.path.exists(log_filename):
    with open(log_filename, 'w') as file:
        pass  # The file is now created.
    
clear_logger() # Clear the logger before reconfiguring it
configure_logging(log_filename, log_file_debug_level="INFO", console_debug_level="INFO")

In [ ]:
import seaborn as sns
import ast
from tqdm.notebook import tqdm

### Define the demographic model

The stdpopsim package maintains demography models and genetic maps. Check the documentation for the demograhy of your choice. You can also build your own demography model.

In [ ]:
import stdpopsim

species = stdpopsim.get_species("HomSap")

# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4788123/
# mutation_rate = 1.4e-8
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1461236/
# mutation_rate = 2.5e-8
# The value used by stdpopsim
mutation_rate = 1.29e-08

# Demographic model for American admixture, taken from Browning et al. 2011
# https://popsim-consortium.github.io/stdpopsim-docs/stable/catalog.html#sec_catalog_homsap_models_americanadmixture_4b11
# populations: ADMIX, AFR, ASIA, EUR
# to build this model, see https://tskit.dev/msprime/docs/stable/demography.html#sec-demography-examples-admixture
demography_model1 = species.get_demographic_model("AmericanAdmixture_4B11").model

# Demographic model for 4 population out of Africa
# https://popsim-consortium.github.io/stdpopsim-docs/stable/catalog.html#sec_catalog_homsap_models_outofafrica_4j171
# https://doi.org/10.1534/genetics.117.200493
# demography_model2 = species.get_demographic_model("OutOfAfrica_4J17").model

# Demographic model for African-Americans from Boyko et al 2008
# https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1000083
# https://popsim-consortium.github.io/stdpopsim-docs/stable/catalog.html#sec_catalog_homsap_models_africa_1b08
# popultions: African_Americans
# demography_model3 = species.get_demographic_model("Africa_1B08").model

In [ ]:
# Run this cell to see the demograhy model defintion

demography_model1

msprime does not provide chromosome-aware simulations. However, as explained in [Simulations without a fixed pedigree](https://tskit.dev/msprime/docs/latest/ancestry.html#simulations-without-a-fixed-pedigree): <blockquote>[Nelson et al. 2020](https://doi.org/10.1371/journal.pgen.1008619) describes a few cases where simulating multiple chromosomes using the DTWF is important for accurately modeling shared patterns of variation between samples. In large simulated cohorts, where many samples can be close relatives, we need the DTWF model and multiple chromosomes to obtain the correct distribution of the number and total length of shared IBD segments. We similarly require multiple chromosomes and the DTWF model to get the correct ancestry patterns for very recently admixed populations.</blockquote>

Because we are simulating both close relatives (and sometimes samples with ancestors from multiple populations), we need chromosome-aware simulations. The following rate map takes that into account.

In [ ]:
import stdpopsim
import msprime
import numpy as np
import math

species = stdpopsim.get_species("HomSap")
genetic_map = species.get_genetic_map("HapMapII_GRCh38")
# genetic_map = species.get_genetic_map("PyrhoYRI_GRCh38")

which_chromosome = list(range(1, 23))
# which_chromosome = [21, 22]

# set the recombination rate in the base pair segments separating chromosomes to log(2)
# see https://tskit.dev/msprime/docs/latest/ancestry.html#example-2-simulating-without-a-pedigree
"""
The effectively reduces the recombination probability
across the gap created by the break. However, it's
important to note that this approach is an approximation
and may not completely prevent recombination across the gap.
"""
r_break = math.log(2)

map_positions = []
map_rates = []
chrom_positions = [0]
# https://www.ncbi.nlm.nih.gov/grc/human/data
genetic_map_positions = {} # as a check for the chromosome positions when converting to multiple chromosomes
conversion_dict = {} # used to convert the chromosome positions to the original positions in the VCF file
assembly_len = 0

for chromosome in which_chromosome:

    # get_chromosome_map
    rate_map = genetic_map.get_chromosome_map(f"chr{chromosome}")
    # genetic_map_positions is a check for the chromosome positions when converting to multiple chromosomes in vcf
    genetic_map_positions[chromosome] = int(rate_map.sequence_length)
    assembly_len += rate_map.sequence_length
    # FIX (potentially): If the rate_map contains a position for a gap (e.g., centromere),
    # replace the given rate with r_break. Potentially, because wouldn't the rate there already
    # reflect a rate aligning with a high recombination rate?

    if chromosome != which_chromosome[0]:
        # print(ending_chromosome)

        # map_positions
        # To make the simulation chromosome-aware, need to create one "long chromosome" of the total needed length

        # Add the positions to map_positions
        # Because the first position of a map is 0, need the + 1 to avoid duplicate positions where maps join
        # print(rate_map.position)
        positions = [x + last_position + 1 for x in rate_map.position]
        # print(positions[0], positions[-1])
        map_positions.extend(positions)

        conversion_dict[(int(last_position + 1), int(last_position + 1 + int(rate_map.sequence_length)))] = chromosome

        # last_position helps to build the sequential "long chromosome" of the total needed length of multiple chromosomes
        # Get the last position in map_positions, which will be the last position in the previous map
        last_position = map_positions[-1]
        chrom_positions.append(int(last_position))

        # Add the map_rates
        # Because each map starts with a span rate of nan, need to add the mean rate to the first span
        # before adding the set of rates for this chromosome to map_rates
        # another option would be to assign it r_break and allow shorter segments from the edges
        shape = rate_map.rate.shape
        rates = rate_map.rate.tolist()
        rates[0] = rate_map.mean_rate
        rates = np.array(rates)
        rates.reshape(shape)
        map_rates.extend(rates)

    else:
        # Add the positions to map_positions
        map_positions.extend(rate_map.position)

        # print(rate_map.position)

        # last_position helps to build the sequential "long chromosome" of the total needed length of multiple chromosomes
        last_position = map_positions[-1]
        chrom_positions.append(int(last_position))

        # Add the map_rates
        map_rates.extend(rate_map.rate)
        conversion_dict[(int(0), int(last_position))] = chromosome


    if chromosome != which_chromosome[-1]:
        map_rates.append(r_break)

rate_map = msprime.RateMap(position=map_positions, rate=map_rates)
assembly_len_maps = int(rate_map.sequence_length)

print(f"The assembly length is {assembly_len_maps:,} bp.")
print(genetic_map_positions)
print(conversion_dict)

With some additional work, you could modify this code to use sex specific genetic maps and/or simulate individuals by sex for the sex chromosomes.

In [ ]:
# import pytz
# print('the supported timezones by the pytz module:',
#       pytz.all_timezones, '\n')
%matplotlib inline
CST = timezone("US/Central")
Ghana_Time = timezone("Africa/Accra")

### msprime pedigree builder

In [ ]:
ped_sim_fam = pd.read_csv(os.path.join(results_directory, "pedigree.fam"), header=0, sep="\t")
ped_sim_fam.columns = ["ind", "parent0", "parent1", "sex"]

# Convert the 'ind' column to integer type
ped_sim_fam['ind'] = ped_sim_fam['ind'].astype(int)

# Replace 0 values with -1 in the "parent0" and "parent1" columns
ped_sim_fam["parent0"] = ped_sim_fam["parent0"].replace(0, -1)
ped_sim_fam["parent1"] = ped_sim_fam["parent1"].replace(0, -1)
ped_sim_fam.head()

In [ ]:
import numpy as np

def assign_generations(ped_df):
    # Initialize generations
    ped_df['generation'] = np.where((ped_df['parent0'] == -1) & (ped_df['parent1'] == -1), 0, -1)
    
    # Function to get generation
    def get_generation(ind):
        row = ped_df.loc[ped_df['ind'] == ind]
        if row['generation'].values[0] != -1:
            return row['generation'].values[0]
        parent0_gen = get_generation(row['parent0'].values[0]) if row['parent0'].values[0] != -1 else 0
        parent1_gen = get_generation(row['parent1'].values[0]) if row['parent1'].values[0] != -1 else 0
        gen = max(parent0_gen, parent1_gen) + 1
        ped_df.loc[ped_df['ind'] == ind, 'generation'] = gen
        return gen
    
    # Assign generations
    for ind in ped_df['ind']:
        get_generation(ind)
    
    # Update spouse generations
    for _, row in ped_df.iterrows():
        if row['parent0'] == -1 and row['parent1'] == -1:
            children = ped_df[(ped_df['parent0'] == row['ind']) | (ped_df['parent1'] == row['ind'])]
            if not children.empty:
                child_gen = children['generation'].min()
                ped_df.loc[ped_df['ind'] == row['ind'], 'generation'] = child_gen - 1
    
    # Reverse generations
    max_gen = ped_df['generation'].max()
    ped_df['generation'] = max_gen - ped_df['generation']
    
    return ped_df


# Apply the function
ped_sim_fam = assign_generations(ped_sim_fam)

# Display the result
ped_sim_fam.head(20)

In [ ]:
# This function converts the text pedigree we created to a msprime pedigree
import tskit 

text_pedigree = ped_sim_fam.copy()

def add_individuals_to_pedigree(pb, text_pedigree):
    """
    msprime pedigree builder.
    Loops through each individual in text pedigree and adds the individual to msprime pedigree with:
    parents, time, population, and metadata of individual_name from text pedigree.
    """
    # Dictionary linking text_pedigree ids to msprime ids
    txt_ped_to_tskit_key = {}

    # For each individual in the genealogy
    for i in text_pedigree.index:
        ind_id = text_pedigree["ind"][i]
        parent0_id = text_pedigree["parent0"][i]
        parent1_id = text_pedigree["parent1"][i]
        ind_time = text_pedigree["generation"][i]
        pop = "AFR"

        # Tells msprime to simulate only generation 0 in the sample
        include_sample = ind_time == 0

        # ped-sim either gives both parents or no parents
        if parent0_id == -1 and parent1_id == -1:
            parent = pb.add_individual(
                time=ind_time,
                population=pop,
                is_sample=include_sample,
                metadata={
                    "individual_name": str(ind_id),
                    "parent0_name": int(parent0_id),
                    "parent1_name": int(parent1_id),
                    "generation": int(ind_time),
                    "population": str(pop)
                }
            )
            txt_ped_to_tskit_key[ind_id] = parent
        else:
            parent0 = txt_ped_to_tskit_key[parent0_id] if parent0_id != -1 else tskit.NULL
            parent1 = txt_ped_to_tskit_key[parent1_id] if parent1_id != -1 else tskit.NULL
            child = pb.add_individual(
                time=ind_time,
                population=pop,
                parents=[parent0, parent1],
                is_sample=include_sample,
                metadata={
                    "individual_name": str(ind_id),
                    "parent0_name": int(parent0_id),
                    "parent1_name": int(parent1_id),
                    "generation": int(ind_time),
                    "population": str(pop)
                }
            )
            txt_ped_to_tskit_key[ind_id] = child

    return pb, txt_ped_to_tskit_key

# Set so that the follow-up recapitation simulations will use the defined
# demography model. Necessary when drawing founders from multiple populations
pb = msprime.PedigreeBuilder(
    demography_model1,
    individuals_metadata_schema=tskit.MetadataSchema.permissive_json()
    )

# Build out the pedigree using input text pedigree
pb, txt_ped_to_tskit_key = add_individuals_to_pedigree(pb, text_pedigree)

# Set the initial state of tree sequence
pedigree = pb.finalise(sequence_length = assembly_len_maps)

print(pedigree)

### Plot the pedigree

Now we plot the pedigree as a sanity check to make sure that it looks like the pedigree we intended. Check your results directory to see more details in the diagram.

In [ ]:
def draw_pedigree(pedigree):
    fig = plt.figure(num=None, figsize=(250, 100), dpi=100)
    G = nx.DiGraph()

    for ind_id, table_row in enumerate(pedigree.individuals):
        # get info from the first node of two nodes for each individual
        node_info = pedigree.nodes[pedigree.nodes.individual == ind_id]
        time = int(node_info.time[0])
        pop = node_info.population[0]

        # add the individual as the graph node
        G.add_node(ind_id, time=time, population=pop)
        for p in table_row.parents:
            if p != tskit.NULL:
                G.add_edge(ind_id, p)

    pos = nx.multipartite_layout(G, subset_key="time", align="horizontal")
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    node_colors = [colors[node_attr["population"]] for node_attr in G.nodes.values()]

    nx.draw_networkx(G, pos, node_size=3000, with_labels=True, node_color=node_colors,
            bbox=dict(facecolor="none", edgecolor='black', boxstyle='round, pad=0.2'))

    filename = f"diagram_msprime_pedigree.svg"
    plt.title("msprime Pedigree")
    plt.savefig(os.path.join(results_directory, filename), bbox_inches = 'tight', pad_inches = 0)

    return G

G = draw_pedigree(pedigree)

### Simulate the tree sequences

In [ ]:
from datetime import datetime
from time import process_time

CST = timezone("US/Central")
Ghana_Time = timezone("Africa/Accra")

# Set the number of replicates, shows variance
# Sequential runs
# num_replicates = os.cpu_count() - 1
num_replicates = 100

t1_start = process_time()

now_est = datetime.now(Ghana_Time)
print("Checkpoint 1: Simulation with the Fixed Pedigree - ", now_est.strftime("%B %d, %-I:%M:%S %p %Z"))

ped_ts_replicates = msprime.sim_ancestry(
    initial_state = pedigree,
    recombination_rate = rate_map,
    model = "fixed_pedigree",
    record_provenance=False,
    discrete_genome = True,
    num_replicates = num_replicates,
    random_seed = 1093871
    )


for replicate_index, ts in enumerate(ped_ts_replicates):

    now_est = datetime.now(Ghana_Time)
    print(f"Checkpoint 2: for ts in ancestry_reps step - {replicate_index}", now_est.strftime("%B %d, %-I:%M:%S %p %Z"))

    completed_ts = msprime.sim_ancestry(
        initial_state = ts,
        recombination_rate = rate_map,
        demography = demography_model1,
        model=[
            msprime.DiscreteTimeWrightFisher(duration=25),
            msprime.StandardCoalescent(),
        ],
        record_provenance=False
        )   
    mutated_ts = msprime.sim_mutations(
        completed_ts, 
        rate=mutation_rate)
    
    # Save the tree sequence
    ts_filename = f"{results_directory}/msprime_tree_replicate_{replicate_index}.trees"
    mutated_ts.dump(ts_filename)

    now_est = datetime.now(Ghana_Time)
    print(f"Checkpoint 3: for ts in ancestry_reps step - {replicate_index}", now_est.strftime("%B %d, %-I:%M:%S %p %Z"))




# remove centromere
# ts = ts.delete_intervals(intervals = centromere_intervals)
# modify sequence length to include `right` telomere

# # Remove centromeres from the simulated tree sequence
# ts = ts.delete_intervals(intervals=centromere_intervals)

# # Modify sequence length to include the right telomere
# telomere_length = 10000  # Adjust the telomere length as needed
# ts = ts.rtrim()
# ts = ts.ltrim(telomere_length)

t1_stop = process_time()

runtime = t1_stop - t1_start
hours = runtime // 3600
minutes = (runtime % 3600) // 60
seconds = (runtime % 3600) % 60

print(f"Runtime: {hours:.0f}:{minutes:.0f}:{seconds:.0f}\n")

In [ ]:

### SEE .simplify()

import numpy as np

def compute_tmrca_for_pairs(ts):
    num_samples = ts.num_samples
    tmrca_matrix = np.zeros((num_samples, num_samples))
    
    for tree in ts.trees():
        for i in range(num_samples):
            for j in range(i + 1, num_samples):  # Iterate over pairs
                # Get the TMRCA for this pair in this tree
                tmrca = tree.time(tree.mrca(i, j))
                # Update the TMRCA matrix if this TMRCA is older
                tmrca_matrix[i, j] = max(tmrca_matrix[i, j], tmrca)
                tmrca_matrix[j, i] = tmrca_matrix[i, j]  # Symmetric matrix
    
    return tmrca_matrix

# Example usage:
# ts = msprime.sim_ancestry(samples=10, sequence_length=1e6, random_seed=42)
# tmrca_matrix = compute_tmrca_for_pairs(ts)


In [ ]:
import concurrent.futures

def worker(seeds):
    ancestry_seed, mutation_seed = seeds
    ts = msprime.sim_ancestry(
        samples=10,
        population_size=100,
        sequence_length=1_000,
        random_seed=ancestry_seed)
    ts = msprime.sim_mutations(ts, rate=1e-6, random_seed=mutation_seed)
    return ts.segregating_sites(span_normalise=False, mode="site")

num_replicates = 100
rng = np.random.RandomState(42)
seeds = rng.randint(1, 2**31, size=(num_replicates, 2))

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(worker, seeds)
    S = np.array(list(results))

(np.mean(S), np.var(S))

A tree sequence is a series of trees that describe the ancestry of different genomic segments. Each tree represents the relationships between individuals for a specific segment of the genome.

### Get Ground Truth

At this point, you have your tree sequences in ped_des_ts_list. You need to create ground truth data to train your machine learning models. You need each node pair's genetic genealogical relationship, mrca, tmrca, segment summaries (e.g., number of segments shared), and segment details (including the segment's mrca and tmrca).

To prepare for future use with tsinfer (from the developers of msprime), you may also consider how to use the IBD segment information to locate the relevant tree sequence and graph a family tree.

I have provided some code snippets below to help you get started.

In [ ]:
import tskit

def detail_segments_to_dataframe(detail_segments):
    data = []
    for node_pair, segment_list in detail_segments.items():
        for segment in segment_list:
            data.append({
                'node1': node_pair[0],
                'node2': node_pair[1],
                'left': segment.left,
                'right': segment.right,
                'segment_mrca': segment.node,
                'span (Mbp)': round(segment.span / 3e6, 1)
            })
    return pd.DataFrame(data)

# Function to get MRCA and TMRCA for a pair of nodes
def get_mrca_info(tree, node1, node2):
    mrca = tree.mrca(node1, node2)
    if mrca != tskit.NULL:
        tmrca = tree.time(mrca)
        return mrca, tmrca
    return None, None

# Function to get ancestors of a node
def get_ancestors(tree, node):
    ancestors = []
    current = tree.parent(node)
    while current != tskit.NULL:
        ancestors.append(current)
        current = tree.parent(current)
    return ancestors

# Function to get descendants of a node
def get_descendants(tree, node):
    return list(tree.descendants(node))

for ts in ped_des_ts_list:
    
    summary_segments = ts.ibd_segments(min_span=3e6, max_time=25, store_pairs=True, store_segments=True)
    print(summary_segments)
    detail_segments = ts.ibd_segments(min_span=3e6, max_time=25, store_pairs=True, store_segments=True)
    detail_df = detail_segments_to_dataframe(detail_segments)
    print("IBD segments (first 5 segments)")
    print(detail_df.head())
    print("\n")
    
    # Analyze IBD segments and corresponding trees
    for node_pair, segment_list in detail_segments.items():
        print(f"IBD segments for node pair {node_pair[0]}, {node_pair[1]}:")
        
        # Iterate over the segments for this pair of nodes
        for segment in segment_list:
            tree = ts.at(segment.left)  # Get the tree at the start of the IBD segment
            
            # Get MRCA and TMRCA
            mrca, tmrca = get_mrca_info(tree, node_pair[0], node_pair[1])
            print(f"MRCA: {mrca}, TMRCA: {tmrca}")
            
            # Get ancestors and descendants for each node in the pair
            for node in node_pair:
                ancestors = get_ancestors(tree, node)
                # descendants = get_descendants(tree, node)
                print(f"Node {node}:")
                print(f"  Ancestors: {ancestors}")
                # print(f"  Descendants: {descendants}")
            
            print("---")  # Separator between segments
    
    print("===")  # Separator between tree sequences